# Install TileDB.CSharp nuget package

In [ ]:
#r "nuget:TileDB.CSharp"

# Create a dense array

## Create dimensions

In [ ]:
var ctx = TileDB.CSharp.Context.GetDefault();
var dom = new TileDB.CSharp.Domain(ctx);
var dim1 = TileDB.CSharp.Dimension.Create<int>(ctx,"rows",1,4,4);
var dim2 = TileDB.CSharp.Dimension.Create<int>(ctx,"cols",1,4,4);
dom.AddDimension(dim1);
dom.AddDimension(dim2);

## Create schema

In [ ]:
var schema = new TileDB.CSharp.ArraySchema(ctx,TileDB.CSharp.ArrayType.Dense);
schema.SetDomain(dom);
var attr1 = new TileDB.CSharp.Attribute(ctx,"a",TileDB.CSharp.DataType.Int32);
schema.AddAttribute(attr1);

## Delete the array if it exists

In [ ]:
string array_uri = "test_dense_array";
var vfs = new TileDB.CSharp.VFS(ctx);
if(vfs.IsDir(array_uri))
{
    vfs.RemoveDir(array_uri);
}

## Create array

In [ ]:
TileDB.CSharp.Array.Create(ctx,array_uri,schema);

# Write the array

## Initialize the data to write

In [ ]:
var data = new int[16];
for(int i=0;i<data.Length;++i)
{
    data[i] = i+1;
}

## Open the array for write

In [ ]:
using (var array_write = new TileDB.CSharp.Array(ctx,array_uri)) {
  array_write.Open(TileDB.CSharp.QueryType.Write);
  var query_write = new TileDB.CSharp.Query(ctx,array_write);
  query_write.SetLayout(TileDB.CSharp.LayoutType.RowMajor);
  query_write.SetDataBuffer<int>("a",data);
  query_write.Submit();
  array_write.Close();
}

# Read the array

## Initialize subarray and read_data

In [ ]:
var subarray = new int[4] {1,2,2,4};
var data_read = new int[6];

## Open array and read

In [ ]:
using (var array_read = new TileDB.CSharp.Array(ctx,array_uri)) 
{
    array_read.Open(TileDB.CSharp.QueryType.Read);
    var query_read = new TileDB.CSharp.Query(ctx,array_read);
    query_read.SetLayout(TileDB.CSharp.LayoutType.RowMajor);
    query_read.SetSubarray<int>(subarray);
    query_read.SetDataBuffer<int>("a",data_read);
    query_read.Submit();
    array_read.Close();
}

## Print query result

In [ ]:
System.Console.WriteLine("query result:");
System.Console.WriteLine(String.Join(" ", data_read));

# Clean up

In [ ]:
if(vfs.IsDir(array_uri))
{
    vfs.RemoveDir(array_uri);
}